In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from urllib.parse import quote_plus

In [30]:
def crawl_naver_news(keyword, start_date, end_date):
    news_data = []
    base_url = "https://search.naver.com/search.naver"
    date_format = "%Y.%m.%d"
    
    while start_date <= end_date:
        params = {
            'where': 'news',
            'query': keyword,
            'sm': 'tab_opt',
            'sort': '0',
            'photo': '0',
            'field': '0',
            'reporter_article': '',
            'pd': '3',
            'ds': start_date.strftime(date_format),
            'de': start_date.strftime(date_format),
            'docid': '',
            'nso': 'so:r,p:from{}to{},a:all'.format(start_date.strftime("%Y%m%d"), start_date.strftime("%Y%m%d")),
            'start': 1,
        }
        
        response = requests.get(base_url, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.select('a.news_tit')
        
        for item in news_items:
            title = item.get('title')
            url = item.get('href')
            news_data.append({'title': title, 'url': url})
        
        # 한 페이지에 모든 결과가 나올 것으로 예상되지만, 필요에 따라 페이지네이션을 처리하는 로직 추가 가능
        start_date += pd.Timedelta(days=1)  # 다음 날짜로 이동

    return pd.DataFrame(news_data)

keyword = "경제, 금융, 위기"
start_date = pd.to_datetime("2008-01-01")
end_date = pd.to_datetime("2008-12-31")
df_news = crawl_naver_news(keyword, start_date, end_date)
print(df_news)

                                       title  \
0                  "위기오면 실력차 날것… 수익성 제고 승부수"   
1               [2008 신년특집] 금융강국의 길 지주사에 달렸다   
2                  2008년 ITㆍ금융ㆍ건설 '新트로이카' 뜬다   
3        [2008 대예측-경제분야] 경제 리더십 발휘 성장엔진 재가동을   
4         <일문일답>權부총리 "한국경제 실적, 국민인식과 단절 아쉽다"   
..                                       ...   
160                美 금융권 '감원한파'…베어스턴스 8천명 감원   
161  [한상춘의 '국제경제 읽기'] 美모기지 사태 '바닥론'과 新유동성 장세   
162                       미국경제 ‘장기 침체’ 주장 근거   
163     잭 웰치 부부의 성공 어드바이스 <50> 경제 위기에서 살아 남기   
164                   李대통령 "물가안정이 7%성장보다 시급"   

                                                   url  
0    http://www.moneytoday.co.kr/view/mtview.php?ty...  
1    http://www.fnnews.com/view?ra=Sent0701m_01A&co...  
2    http://www.hankyung.com/news/app/newsview.php?...  
3    http://www.heraldbiz.com/SITE/data/html_dir/20...  
4    https://n.news.naver.com/mnews/article/003/000...  
..                                                 ...  
160  http://seoulfn.com/

In [27]:
# 데이터프레임을 CSV 파일로 저장
df_news.to_csv('/home/verlo/dev_ws/EDA/data/naver_news.csv', index=False, encoding='utf-8-sig')